In [1]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import dataframefunctions
import pandas as pd

In [7]:
evaluation = pd.DataFrame([[1, 2]],
                              columns=[["a", "a"], ["Mean", "Standard deviation"]],
                              index=["Dataset"])

In [8]:
evaluation.head()

a                   
        Mean Standard deviation
Dataset    1                  2

In [2]:
iris_dataset = pd.read_csv("..\Datasets\iris.csv")

In [3]:
# TODO
def render_run_predictions(dataframe):
    label_name = list(dataframe.columns)[-1]
    x_prepared, y_prepared = preprocess_dataset(dataframe, label_name)
    x_dataset = pd.DataFrame(x_prepared, columns=dataframe.columns[:-1], index=dataframe.index)
    y_dataset = pd.DataFrame(y_prepared, columns=[label_name], index=dataframe.index)
    return x_dataset, y_dataset


def preprocess_dataset(dataframe, label_name):

    x = dataframe.drop(label_name, axis=1)
    y = dataframe[label_name].copy()

    x_prepared = standard_data_preprocessing(x)
    y_prepared = y if not is_categorical(y) else standard_label_preprocessing(y)

    return x_prepared, y_prepared


def standard_data_preprocessing(train_data):

    numerical_att = get_numeric_columns(train_data)
    categorical_att = get_categorical_columns(train_data)
    num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

    cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="most_frequent")),
        ('oneHot', OneHotEncoder()),
    ])

    full_pipeline = ColumnTransformer([
        ("num", num_pipeline, numerical_att),
        ("cat", cat_pipeline, categorical_att),
    ])

    return full_pipeline.fit_transform(train_data)


def is_categorical(column):
    return column.dtype.name == 'object'


def standard_label_preprocessing(y):
    lc = LabelEncoder().fit(y)
    return lc.transform(y)

def get_categorical_columns(df):
    return list(df.select_dtypes(exclude=['number']).columns.values)

def get_numeric_columns(df):
    return list(df.select_dtypes(['number']).columns.values)

In [4]:
a, b = render_run_predictions(iris_dataset)

In [38]:
b.head()

,variety
0,0
1,0
2,0
3,0
4,0


In [5]:
X_train, X_test, y_train, y_test = train_test_split(a, b, test_size=0.2, random_state=5)

In [6]:
model = XGBClassifier() 

model.fit(X_train, y_train)

C:\Users\Andrea Moscatelli\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Andrea Moscatelli\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [7]:
y_pred = model.predict(X_test) 

predictions = [round(value) for value in y_pred]

In [8]:
accuracy = accuracy_score(y_test, predictions) 

print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.00%
